# Representación semántica y búsqueda
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

## Usando la nube de puntos

La representación de nube de puntos puede usarse directamente sin realizar prototipos, sin embargo, es posible que dada su complejidad, dimensión intrinseca, tenga poca utilidad en grandes volumenes de información. Por ejemplo, es posible utilizar la distancia de [Hausdorff](https://en.wikipedia.org/wiki/Hausdorff_distance); que entre dos nubes de puntos (documentos para este caso) esta definida como sigue:

$$ H_d(U, V) = \max \left\{ \max_{u \in U} \min_{x \in V} d(u, x), \max_{v \in V} \min_{x \in U} d(v, x) \right\} $$

Donde $\textsf{nn}$ encuentra el vecino cercano del primer argumento en el segundo. Podemos ver la intuición de esta distancia, analizando las partes de la expresión. 
La diferencia entre ambos conjuntos se representa con el máximo de las distancias mínimas. Note que esta parte es idéntica a resolver una búsqueda de vecinos cercanos. La parte más externa de la expresión se repite para los dos conjuntos para preservar la simétria de la función.

De la misma forma, es posible obtener variaciones de interés como considerar todas las distancias cercanas en lugar de solo las máximas, lo cual puede reducir formas caprichosas de _outlayers_.

$$ H^+_d(U, V) = \max \left\{ \sum_{u \in U} \min_{x \in V} d(u, x), \sum_{v \in V} \min_{x \in U} d(v, x) \right\}$$

La información local de una representación de bolsa de palabras también puede ser aprovechada, por ejemplo, añadiendo información de $\textsf{TF}$

$$ H^\textsf{TF}_d(U, V) = \max \left\{ \sum_{u \in U} {\textsf{TF}(u, U) \min_{x \in V} d(u, x)}, \sum_{v \in V} {\textsf{TF}(v, V) \min_{x \in U} d(v, x)} \right\} $$

# Notas adicionales
La versión de $H_d$ que usa la probabilidad de término $\textsf{pt}$ es similar a la aproximación 2 listada en [@KSKW2015]. Los prototipos son similares a la aproximación 1 del mismo artículo. En el artículo se presenta la función de distancia _word mover's distance_ (WMD) que es una adecuación de la Earths mover's distance (EMD) [PW2009]. EMD es la solución óptima a un problema de transportación, la WMD es la adaptación teniendo en cuenta embeddings de palabras.

# Ejemplo


In [1]:

using Pkg
Pkg.activate(".")
!isfile("Manifest.toml") && Pkg.add([
    PackageSpec(name="SimilaritySearch", version="0.9"),
    PackageSpec(name="TextSearch", version="0.12"),
    PackageSpec(name="Plots"),
    PackageSpec(name="KNearestCenters"),
    PackageSpec(name="HypertextLiteral"),
    PackageSpec(name="CSV"),
    PackageSpec(name="DataFrames"),
    PackageSpec(name="Embeddings", version="0.4")
])

using SimilaritySearch, TextSearch, Plots, KNearestCenters, LinearAlgebra, Embeddings, HypertextLiteral, CSV, DataFrames, Base64, Random
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
]status

      Status `~/IR-2022/Unidades/Project.toml`
  [944b1d66] CodecZlib v0.7.0
  [ac1192a8] HypertextLiteral v0.9.4
  [b20bd276] InvertedFiles v0.4.1 `../../Research/InvertedFiles.jl`
  [682c06a0] JSON v0.21.3
  [4dca28ae] KNearestCenters v0.7.1
  [8ef0a80b] Languages v0.4.3
  [eb30cadb] MLDatasets v0.7.3
  [053f045d] SimilaritySearch v0.9.4 `../../Research/SimilaritySearch.jl`
  [fb8f903a] Snowball v0.1.0
  [2913bbd2] StatsBase v0.33.18
  [7f6f6c8a] TextSearch v0.12.5 `../../Research/TextSearch.jl`


In [3]:
function knn(index, labels, q, k)
    res = KnnResult(k)
    search(index, q, res)
    mode(labels[idview(res)])
end

function mymode(c, labels, f)
    n = length(c)
    empty!(f)
    for id in c
        id == 0 && break  # searchbatch stores zeros at the end of the result when the result set is smaller than the required one
        l = labels[id]
        f[l] = get(f, l, 0) + 1
    end
    
    if length(f) == 0
        rand(labels)
    else
        argmax(last, f) |> first
    end
end

function knn(I, labels)
    f = Dict{eltype(labels), Int}()
    [mymode(c, labels, f) for c in eachcol(I)]
end

function scores(gold, pred)
    s = classification_scores(gold, pred)
    (macrof1=s.macrof1, macrorecall=s.macrorecall, accuracy=s.accuracy)
end

scores (generic function with 1 method)

In [4]:
function text_model_and_vectors(
        corpus, vocab;
        localweighting=TpWeighting(),
        globalweighting=BinaryGlobalWeighting(),
        nlist=[1],
        qlist=[],
        slist=[],
        group_usr=true,
        group_url=true,
        group_num=true,
        del_diac=true,
        lc=true
    )

    voc = Vocabulary(length(corpus))
    for v in vocab
       push!(voc, v)
    end
    
    textconfig = TextConfig(; group_usr, group_url, del_diac, lc, group_num, nlist, qlist, slist)
    tokenize_and_append!(voc, textconfig, corpus)
    model = VectorModel(globalweighting, localweighting, voc)
    (; textconfig, model, voc)
end


text_model_and_vectors (generic function with 1 method)

In [5]:
function embeddings()
    embfile = "../data/MX.vec"
    !isfile(embfile) && download("http://geo.ingeotec.mx/~sadit/regional-spanish-models/MX.vec", embfile)
    emb = load_embeddings(FastText_Text, embfile)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end
    
    (; X=emb.embeddings, vocab=emb.vocab)
end

function vectorize_as_prototype!(c, text, E, T)
    x = vectorize(T.model, T.textconfig, text; normalize=false)
    if length(x) == 1 && haskey(x, 0)
        #@warn "empty vector $(Int(i)) selecting a random vector for it " # $(corpus[i])
        rand!(c)
    else
        for (id, weight) in x
            id == 0 && continue
             c .= c .+ weight .* view(E.X, :, id)
        end
    end
    
    normalize!(c)
end

function vectorize_corpus_as_prototypes(corpus, E, T)
    dim = size(E.X, 1)
    n = length(corpus)
    C = zeros(Float32, dim, n)
    # Threads.@threads
    for i in 1:n
        vectorize_as_prototype!(view(C, :, i), corpus[i], E, T)
    end
    
    C
end

function create_index(db)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(0.9)))
    optimize!(index, MinRecall(0.9))
    index
end

create_index (generic function with 1 method)

In [6]:
E = embeddings()
D = CSV.read("../data/spanish-news-in-twitter.csv", DataFrame; delim=',')
D[:, :text] = String.(base64decode.(D.text))
dropmissing!(D)
T = text_model_and_vectors(D.text, E.vocab)

(textconfig = TextConfig(true, false, false, true, true, true, false, true, Int8[], Int8[1], Skipgram[], IdentityTokenTransformation()), model = {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=447763, train-n=41998, maxoccs=59524}, voc = Vocabulary(["</s>", "_usr", "que", "de", ",", ".", "y", "a", "la", "no"  …  "#alejandromoreno", "tantitititiititiitititititititititititititititititititititititita", "multiejecucion", "#marcoscastellanos", "devolviendola", "senadoz", "#alitomoreno", "dastis", "javerianizacion", "#bundesliga"], Int32[0, 25844, 20315, 59524, 31605, 21433, 17403, 20211, 34402, 6002  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Int32[0, 25844, 20315, 59524, 31605, 21433, 17403, 20211, 34402, 6002  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Dict{String, UInt32}("alaann" => 0x0005f201, "coprolito" => 0x0005f0e5, "alissonrojas" => 0x00018f

In [7]:
@time C = vectorize_corpus_as_prototypes(D.text, E, T)

LoadError: BoundsError: attempt to access 300×438136 Matrix{Float32} at index [1:300, 438137]

### Se crea el índice métrico

In [8]:
@time index = create_index(MatrixDatabase(C))

LoadError: UndefVarError: C not defined

### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [9]:
let k = 32
    t1 = @elapsed I, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    t2 = @elapsed gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.screen_name)
    s1 = scores(labels, knn(I, labels))
    s2 = scores(labels, knn(gI, labels))
    
    @htl """
    <div>macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

LoadError: UndefVarError: index not defined

### Búsqueda y presentación de los resultados

In [10]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.screen_name[id])</td><td> $(D.text[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [11]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

LoadError: UndefVarError: index not defined

In [12]:

display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")
for i in 1:3
    for qid in rand(1:length(D.text))
        search_and_display(index, D.text[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

LoadError: UndefVarError: index not defined

# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.- Defina $H^{pt}_d$, donde $pt$ es la probabilidad de ocurrencia de término.
- Implemente $H^\textsf{TF}_d$ y $H^\textsf{pt}_d$ de manera secuencial y con un índice métrico. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. El reporte deberá contener un ensayo de [@KSKW2015] como introducción. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
